In [ ]:
import numpy as np
import torch
from torch import nn

from model_tools import *

In [ ]:
cut_window = (128, 128)
test_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_test.npy')

# Load and preprocess data
# Test
dataset_creator_test = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=test_indices, cut_window=None,
    create_shared_memory=False
)
dataset_test = HsiDataloaderCutter(
    images=dataset_creator_test.images, masks=dataset_creator_test.masks,
    augmentation=test_augmentation,
    shuffle_data=False, cut_window=None,
)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

net = MySuperNetLittleInput(in_f=17, out_f=17)
net.eval()
with torch.no_grad():
    _ = net(torch.randn(1, 17, cut_window[0], cut_window[1]))
net.train()

In [ ]:
model = NnModel.load_from_checkpoint(
    "model-epoch=447-mean_iou=0.61.ckpt",
    loss=nn.CrossEntropyLoss(), model=net
)
net.eval()


In [ ]:
outputs = []
for data in test_loader:
    outputs.append(data)

In [ ]:
pred_tensor, target_tensor = collect_prediction_and_target(outputs, net, cut_window=cut_window)
target_one_hotted_tensor = torch.nn.functional.one_hot(
    target_tensor, 17 # Num classes
)
# (N, H, W, C) --> (N, C, H, W)
target_one_hotted_tensor = target_one_hotted_tensor.permute(0, -1, 1, 2)

In [ ]:
np.save("target_one_hotted_tensor", target_one_hotted_tensor.detach().numpy())

In [ ]:
np.save("pred_tensor", pred_tensor.detach().numpy())